In [1]:
import requests
import pandas as pd
import json
import datetime
import numpy as np

# Pull Latitude and Longitude from positionstack.com based on geographical user input.

## temporary hack - uncomment and run the cell below once, then recomment

In [2]:
# bool_cols = ['freezing', 'cold', 'warm', 'hot',
#              'not_rainy', 'lightly_rainy', 'very_rainy',
#              'not_snowy', 'lightly_snowy', 'very_snowy',
#              'windy',
#              'low_humidity', 'medium_humidity', 'high_humidity',
#              'clear', 'partly_cloudy', 'cloudy']

# blank_dict = {stat:[] for stat in bool_cols}
# blank_dict['geo'] = []

# hist_stats_df = pd.DataFrame(blank_dict)

# hist_stats_df.to_csv('five_cities.csv')

## temporary hack - enter country, city below, then run-all

In [3]:
country_input = 'jp'
query_input = 'Tokyo'

In [4]:
# # User input: Country
# country_input = (input("Enter a Country using a 2-letter country code (Example: US): "))

In [5]:
# N/A: No need for this extra geographic variable.

# User input: State
# region_input = (input("Enter a region (City, County, State, etc.): "))

In [6]:
# # User input: City

# # HTTP GET Request Parameters:
# # query	[Required] Specify your query as a free-text address, 
# # place name or using any other common text-based location 
# # identifier (e.g. postal code, city name, region name).

# query_input = (input("Enter a location (City, Zip Code, etc.): "))

In [7]:
# Geo reference for adding to final dataframe.
geo_input = country_input  + " | " + query_input
geo_input

'jp | Tokyo'

In [8]:
import http.client, urllib.parse

conn = http.client.HTTPConnection('api.positionstack.com')

params = urllib.parse.urlencode({
    'access_key': '1fbf7de4ce05274e9b9005659970c429',
    
    # Variables get live user input above.
    'country': country_input,
#     'region': region_input,
    'query': query_input,
        
    # "limit" of 1 ensures only 1 Lat/Lon will be passed below.    
    'limit': 1
    })

conn.request('GET', '/v1/forward?{}'.format(params))

res = conn.getresponse()
geo = res.read()
geo_str = (geo.decode('utf-8'))

print(geo.decode('utf-8'))

###########################################################
# Still needs error handling.

{"data":[{"latitude":35.695126,"longitude":139.75391,"type":"locality","name":"Tokyo","number":null,"postal_code":null,"street":null,"confidence":1,"region":"Tokyo","region_code":"TK","county":"Chiyoda","locality":"Tokyo","administrative_area":null,"neighbourhood":null,"country":"Japan","country_code":"JPN","continent":"Asia","label":"Tokyo, Japan"}]}


In [9]:
# Help from AskBCS:
# Result was one long string, that only looked 
# like a dictionary, but wasn't.
# Use "type()" to identify whether or not 
# result is actually a dict.

type(geo_str)

str

In [10]:
# Convert string to json (which is a dictionary).
# geo_json
geo_json = json.loads(geo_str)

print(geo_json)

{'data': [{'latitude': 35.695126, 'longitude': 139.75391, 'type': 'locality', 'name': 'Tokyo', 'number': None, 'postal_code': None, 'street': None, 'confidence': 1, 'region': 'Tokyo', 'region_code': 'TK', 'county': 'Chiyoda', 'locality': 'Tokyo', 'administrative_area': None, 'neighbourhood': None, 'country': 'Japan', 'country_code': 'JPN', 'continent': 'Asia', 'label': 'Tokyo, Japan'}]}


##### Reminder: Double parentheses are functional in the for loops.

In [11]:
# # geo_json_coord = lat and lon combined into a tuple.

# geo_json_coord = geo_json['data']
# latlon_tup = []
# for x in geo_json_coord:
#     latlon_tup.append((x['latitude'], x['longitude']))

In [12]:
# Pull latitude value by itself.

geo_json_lat = geo_json['data']
var_lat = []
for x in geo_json_lat:
    var_lat.append((x['latitude']))
    
var_lat[0]

35.695126

In [13]:
# Pull longitude value by itself.

geo_json_lon = geo_json['data']
var_lon = []
for x in geo_json_lon:
    var_lon.append((x['longitude']))
    
var_lon[0]

139.75391

# Push Latitude and Longitude to Weather API URL, based on start date and stop date from user input.

## Hourly data.

In [14]:
# List of columns, for starter dataset, hourly:

# hourly_columns = ['time',
#              'temperature_2m_degF', 
#              'relativehumidity_2m_perc', 
#              'rain_inch',
#              'snowfall_cm', 
#              'cloudcover_perc', 
#              'windspeed_10m_mph',
#              'winddirection_10m_deg']

#  We pulled data from 2000-01-01 to 2020-12-31.

Original Open Meteo API URL, hourly:

https://archive-api.open-meteo.com/v1/era5?latitude=52.52&longitude=13.41&start_date=2000-01-01&end_date=2020-12-31&hourly=temperature_2m,relativehumidity_2m,rain,snowfall,cloudcover,windspeed_10m,winddirection_10m&timezone=America%2FLos_Angeles&temperature_unit=fahrenheit&windspeed_unit=mph&precipitation_unit=inch

In [15]:
# User input: Start Date
# start_date_input = (input("Enter the start date (YYYY-MM-DD): "))

In [16]:
# User input: End Date
# end_date_input = (input("Enter the end date (YYYY-MM-DD): "))

In [17]:
# Break URL into snippets to be assembled below, hourly:

url_snip_hr_1 = "https://archive-api.open-meteo.com/v1/era5?latitude=" 
url_snip_hr_2 = str(var_lat[0])   # latitude, input from previous json
url_snip_hr_3 = "&longitude="
url_snip_hr_4 = str(var_lon[0])   # longitude, input from previous json



url_snip_hr_5 = "&start_date="
# url_snip_hr_6 = start_date_input   # start date, input from user
url_snip_hr_6 = "2010-01-01"   # --- OR --- static start date, for dummy data

url_snip_hr_7 = "&end_date="
# url_snip_hr_8 = end_date_input   # end date, input from user
url_snip_hr_8 = "2019-12-31"   # --- OR ---  static end date, for dummy data



url_snip_hr_9 = "&hourly="   # elements, hourly
url_snip_hr_10 = "temperature_2m,"   # data element
url_snip_hr_11 = "relativehumidity_2m,"   # data element
url_snip_hr_12 = "rain,"   # data element
url_snip_hr_13 = "snowfall,"   # data element
url_snip_hr_14 = "cloudcover,"   # data element
url_snip_hr_15 = "windspeed_10m,"   # data element
url_snip_hr_16 = "winddirection_10m"   # data element

url_snip_hr_17 = "&timezone=America%2FLos_Angeles"   # time zone
url_snip_hr_18 = "&temperature_unit=fahrenheit"   # temp unit
url_snip_hr_19 = "&windspeed_unit=mph"   # windspeed unit
url_snip_hr_20 = "&precipitation_unit=inch"   # precip unit

In [18]:
# Contatenate the url snippets, hourly.

url_hr_concat_full = \
    url_snip_hr_1 + \
    url_snip_hr_2 + \
    url_snip_hr_3 + \
    url_snip_hr_4 + \
    url_snip_hr_5 + \
    url_snip_hr_6 + \
    url_snip_hr_7 + \
    url_snip_hr_8 + \
    url_snip_hr_9 + \
    url_snip_hr_10 + \
    url_snip_hr_11 + \
    url_snip_hr_12 + \
    url_snip_hr_13 + \
    url_snip_hr_14 + \
    url_snip_hr_15 + \
    url_snip_hr_16 + \
    url_snip_hr_17 + \
    url_snip_hr_18 + \
    url_snip_hr_19 + \
    url_snip_hr_20

In [19]:
print(url_hr_concat_full)

https://archive-api.open-meteo.com/v1/era5?latitude=35.695126&longitude=139.75391&start_date=2010-01-01&end_date=2019-12-31&hourly=temperature_2m,relativehumidity_2m,rain,snowfall,cloudcover,windspeed_10m,winddirection_10m&timezone=America%2FLos_Angeles&temperature_unit=fahrenheit&windspeed_unit=mph&precipitation_unit=inch


In [20]:
# Data comes in as one long string.
weather_hr_str = requests.get(url_hr_concat_full).text

In [21]:
# Convert string to dictionary.
weather_hr_json = json.loads(weather_hr_str)

In [22]:
# Verify that data type is dictionary.
type(weather_hr_json)

dict

In [23]:
# Convert json dictionary into dataframe.

# Standard "pd.json_normalize(weather_json)" doesn't work 
# with these nested levels. Also tried adding "record_path"
# and "meta" tags, but still didn't work.
# Found "from_records" alternative that does work.

# N/A: Code from:
# https://stackoverflow.com/questions/66894920/json-normalise-from-nested-api-to-df-throwing-type-error
# Creates "pivot" table format with all entries as columns, 
# all variables as rows.
# Not what we need, but might be useful for other datasets.
# dftest3 = pd.DataFrame.from_records(weather_json['hourly']).T.reset_index()


# Code from:
# https://stackoverflow.com/questions/66894920/json-normalise-from-nested-api-to-df-throwing-type-error

weather_hourly_raw = pd.DataFrame.from_records(weather_hr_json['hourly'])
weather_hourly_raw

,cloudcover,rain,relativehumidity_2m,snowfall,temperature_2m,time,winddirection_10m,windspeed_10m
0,1,0.0,37,0.0,45.0,2010-01-01T00:00,15.0,8.6
1,0,0.0,48,0.0,41.5,2010-01-01T01:00,21.0,8.1
2,0,0.0,54,0.0,39.3,2010-01-01T02:00,16.0,7.4
3,4,0.0,62,0.0,35.8,2010-01-01T03:00,24.0,3.9
4,1,0.0,64,0.0,34.9,2010-01-01T04:00,20.0,5.2
...,...,...,...,...,...,...,...,...
87643,24,0.0,41,0.0,43.2,2019-12-31T19:00,5.0,4.7
87644,35,0.0,40,0.0,44.8,2019-12-31T20:00,27.0,2.0
87645,8,0.0,40,0.0,46.0,2019-12-31T21:00,120.0,1.8
87646,0,0.0,41,0.0,46.8,2019-12-31T22:00,156.0,2.7


In [24]:
# Add geo input from user as new column, for reference.

weather_hourly_raw['geo'] = geo_input
weather_hourly_raw

,cloudcover,rain,relativehumidity_2m,snowfall,temperature_2m,time,winddirection_10m,windspeed_10m,geo
0,1,0.0,37,0.0,45.0,2010-01-01T00:00,15.0,8.6,jp | Tokyo
1,0,0.0,48,0.0,41.5,2010-01-01T01:00,21.0,8.1,jp | Tokyo
2,0,0.0,54,0.0,39.3,2010-01-01T02:00,16.0,7.4,jp | Tokyo
3,4,0.0,62,0.0,35.8,2010-01-01T03:00,24.0,3.9,jp | Tokyo
4,1,0.0,64,0.0,34.9,2010-01-01T04:00,20.0,5.2,jp | Tokyo
...,...,...,...,...,...,...,...,...,...
87643,24,0.0,41,0.0,43.2,2019-12-31T19:00,5.0,4.7,jp | Tokyo
87644,35,0.0,40,0.0,44.8,2019-12-31T20:00,27.0,2.0,jp | Tokyo
87645,8,0.0,40,0.0,46.0,2019-12-31T21:00,120.0,1.8,jp | Tokyo
87646,0,0.0,41,0.0,46.8,2019-12-31T22:00,156.0,2.7,jp | Tokyo


## Daily data.

In [25]:
# List of columns, for starter dataset, daily:

# daily_columns = ['time',
#            'temperature_2m_degF_max',
#            'temperature_2m_degF_min',  
#            'rain_inch',
#            'snowfall_cm',
#            'precipitation_hours'
#           ]

#  We pulled data from 2000-01-01 to 2020-12-31.

Original Open Meteo API URL, daily:

https://archive-api.open-meteo.com/v1/era5?latitude=52.52&longitude=13.41&start_date=2000-01-01&end_date=2020-12-31&daily=temperature_2m_max,temperature_2m_min,rain_sum,snowfall_sum,precipitation_hours&timezone=America%2FLos_Angeles&temperature_unit=fahrenheit&windspeed_unit=mph&precipitation_unit=inch

In [26]:
# Break URL into snippets to be assembled below, daily:

url_snip_dy_1 = "https://archive-api.open-meteo.com/v1/era5?latitude=" 
url_snip_dy_2 = str(var_lat[0])   # latitude, input from previous json
url_snip_dy_3 = "&longitude="
url_snip_dy_4 = str(var_lon[0])   # longitude, input from previous json



url_snip_dy_5 = "&start_date="
# url_snip_dy_6 = start_date_input   # start date, input from user
url_snip_dy_6 = "2010-01-01"   # --- OR --- static start date, for dummy data

url_snip_dy_7 = "&end_date="
# url_snip_dy_8 = end_date_input   # end date, input from user
url_snip_dy_8 = "2019-12-31"   # --- OR ---  static end date, for dummy data



url_snip_dy_9 = "&daily="   # elements, daily
url_snip_dy_10 = "temperature_2m_max,"   # data element
url_snip_dy_11 = "temperature_2m_min,"   # data element
url_snip_dy_12 = "rain_sum,"   # data element
url_snip_dy_13 = "snowfall_sum,"   # data element
url_snip_dy_14 = "precipitation_hours"   # data element


url_snip_dy_15 = "&timezone=America%2FLos_Angeles"   # time zone
url_snip_dy_16 = "&temperature_unit=fahrenheit"   # temp unit
url_snip_dy_17 = "&windspeed_unit=mph"   # windspeed unit
url_snip_dy_18 = "&precipitation_unit=inch"   # precip unit

In [27]:
# Contatenate the url snippets, hourly.

url_dy_concat_full = \
    url_snip_dy_1 + \
    url_snip_dy_2 + \
    url_snip_dy_3 + \
    url_snip_dy_4 + \
    url_snip_dy_5 + \
    url_snip_dy_6 + \
    url_snip_dy_7 + \
    url_snip_dy_8 + \
    url_snip_dy_9 + \
    url_snip_dy_10 + \
    url_snip_dy_11 + \
    url_snip_dy_12 + \
    url_snip_dy_13 + \
    url_snip_dy_14 + \
    url_snip_dy_15 + \
    url_snip_dy_16 + \
    url_snip_dy_17 + \
    url_snip_dy_18

In [28]:
print(url_dy_concat_full)

https://archive-api.open-meteo.com/v1/era5?latitude=35.695126&longitude=139.75391&start_date=2010-01-01&end_date=2019-12-31&daily=temperature_2m_max,temperature_2m_min,rain_sum,snowfall_sum,precipitation_hours&timezone=America%2FLos_Angeles&temperature_unit=fahrenheit&windspeed_unit=mph&precipitation_unit=inch


In [29]:
# Data comes in as one long string.
weather_dy_str = requests.get(url_dy_concat_full).text

In [30]:
# Convert string to dictionary.
weather_dy_json = json.loads(weather_dy_str)

In [31]:
# Verify that data type is dictionary.
type(weather_dy_json)

dict

In [32]:
# Convert json dictionary into dataframe.

# Standard "pd.json_normalize(weather_json)" doesn't work 
# with these nested levels. Also tried adding "record_path"
# and "meta" tags, but still didn't work.
# Found "from_records" alternative that does work.

# N/A: Code from:
# https://stackoverflow.com/questions/66894920/json-normalise-from-nested-api-to-df-throwing-type-error
# Creates "pivot" table format with all entries as columns, 
# all variables as rows.
# Not what we need, but might be useful for other datasets.
# dftest3 = pd.DataFrame.from_records(weather_json['hourly']).T.reset_index()


# Code from:
# https://stackoverflow.com/questions/66894920/json-normalise-from-nested-api-to-df-throwing-type-error

weather_daily_raw = pd.DataFrame.from_records(weather_dy_json['daily'])
weather_daily_raw

,precipitation_hours,rain_sum,snowfall_sum,temperature_2m_max,temperature_2m_min,time
0,0.0,0.00,0.0,50.9,32.2,2010-01-01
1,0.0,0.00,0.0,49.3,32.0,2010-01-02
2,0.0,0.00,0.0,47.0,33.6,2010-01-03
3,3.0,0.05,0.0,54.5,38.1,2010-01-04
4,1.0,0.00,0.0,51.4,35.9,2010-01-05
...,...,...,...,...,...,...
3647,0.0,0.00,0.0,50.6,37.8,2019-12-27
3648,0.0,0.00,0.0,49.3,36.0,2019-12-28
3649,2.0,0.01,0.0,48.6,41.7,2019-12-29
3650,0.0,0.00,0.0,64.1,41.3,2019-12-30


In [33]:
# Add geo input from user as new column, for reference.

weather_daily_raw['geo'] = geo_input
weather_daily_raw

,precipitation_hours,rain_sum,snowfall_sum,temperature_2m_max,temperature_2m_min,time,geo
0,0.0,0.00,0.0,50.9,32.2,2010-01-01,jp | Tokyo
1,0.0,0.00,0.0,49.3,32.0,2010-01-02,jp | Tokyo
2,0.0,0.00,0.0,47.0,33.6,2010-01-03,jp | Tokyo
3,3.0,0.05,0.0,54.5,38.1,2010-01-04,jp | Tokyo
4,1.0,0.00,0.0,51.4,35.9,2010-01-05,jp | Tokyo
...,...,...,...,...,...,...,...
3647,0.0,0.00,0.0,50.6,37.8,2019-12-27,jp | Tokyo
3648,0.0,0.00,0.0,49.3,36.0,2019-12-28,jp | Tokyo
3649,2.0,0.01,0.0,48.6,41.7,2019-12-29,jp | Tokyo
3650,0.0,0.00,0.0,64.1,41.3,2019-12-30,jp | Tokyo


## aggregating hourlies into dailies for historical summary

In [34]:
weather_hourly_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87648 entries, 0 to 87647
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   cloudcover           87648 non-null  int64  
 1   rain                 87648 non-null  float64
 2   relativehumidity_2m  87648 non-null  int64  
 3   snowfall             87648 non-null  float64
 4   temperature_2m       87648 non-null  float64
 5   time                 87648 non-null  object 
 6   winddirection_10m    87630 non-null  float64
 7   windspeed_10m        87648 non-null  float64
 8   geo                  87648 non-null  object 
dtypes: float64(5), int64(2), object(2)
memory usage: 6.0+ MB


In [35]:
# slice of hourly to work with
hourly_temp_df = weather_hourly_raw[['time', 'cloudcover', 'relativehumidity_2m',
                                     'temperature_2m', 'windspeed_10m']].copy()

In [36]:
# convert the provided ISO string 'time' into a 'pure_date' in python datetime format
# for aggregating and joining with daily data;
# ==>> this should be done above, in intial datacleaning <<==

hourly_temp_df["pure_date"] = hourly_temp_df['time'].map(lambda x: 
                                                     datetime.datetime.fromisoformat(x[0:10]))

In [37]:
def enth(x, n):
    return x.sort_values().iloc[n]

In [38]:
step1_df = \
hourly_temp_df.groupby('pure_date').agg(humid_avg=('relativehumidity_2m', np.mean),
                                        wind_high = ('windspeed_10m', np.max),
                                        cloud_avg=('cloudcover', np.mean),
                                        temp_6= ('temperature_2m', lambda x: enth(x,18)) )
step1_df.head()

,humid_avg,wind_high,cloud_avg,temp_6
pure_date,,,,
2010-01-01,51.916667,8.6,9.500000,45.0
2010-01-02,58.375000,8.6,9.458333,46.0
2010-01-03,56.708333,14.0,39.583333,43.2
2010-01-04,66.208333,10.2,22.958333,45.9
2010-01-05,48.416667,16.1,9.291667,47.9


## slicing the chunk of daily to join

In [39]:
weather_daily_raw.columns

Index(['precipitation_hours', 'rain_sum', 'snowfall_sum', 'temperature_2m_max',
       'temperature_2m_min', 'time', 'geo'],
      dtype='object')

In [40]:
# get the columns we want, rename as per specs

daily_temp_df = weather_daily_raw[['time', 'precipitation_hours', 'rain_sum',
                                   'snowfall_sum', 'temperature_2m_max']].copy()

daily_temp_df.rename(columns={'temperature_2m_max': 'temp_high'}, inplace=True)


In [41]:
# convert the provided ISO string 'time' into a 'pure_date' in python datetime format
# for aggregating and joining with daily data;
# ==>> this should be done above, in intial datacleaning <<==

daily_temp_df["pure_date"] = daily_temp_df['time'].map(lambda x: 
                                                     datetime.datetime.fromisoformat(x[0:10]))

In [42]:
daily_temp_df.drop('time', axis=1, inplace=True)

In [43]:
daily_temp_df.set_index('pure_date', inplace=True)

In [44]:
daily_temp_df.head()

,precipitation_hours,rain_sum,snowfall_sum,temp_high
pure_date,,,,
2010-01-01,0.0,0.00,0.0,50.9
2010-01-02,0.0,0.00,0.0,49.3
2010-01-03,0.0,0.00,0.0,47.0
2010-01-04,3.0,0.05,0.0,54.5
2010-01-05,1.0,0.00,0.0,51.4


In [45]:
step1_df = step1_df.join(daily_temp_df)
step1_df.head()

,humid_avg,wind_high,cloud_avg,temp_6,precipitation_hours,rain_sum,snowfall_sum,temp_high
pure_date,,,,,,,,
2010-01-01,51.916667,8.6,9.500000,45.0,0.0,0.00,0.0,50.9
2010-01-02,58.375000,8.6,9.458333,46.0,0.0,0.00,0.0,49.3
2010-01-03,56.708333,14.0,39.583333,43.2,0.0,0.00,0.0,47.0
2010-01-04,66.208333,10.2,22.958333,45.9,3.0,0.05,0.0,54.5
2010-01-05,48.416667,16.1,9.291667,47.9,1.0,0.00,0.0,51.4


## Step 2: New columns for weather category labels.

In [46]:
step2_df = step1_df.copy()
step2_df

,humid_avg,wind_high,cloud_avg,temp_6,precipitation_hours,rain_sum,snowfall_sum,temp_high
pure_date,,,,,,,,
2010-01-01,51.916667,8.6,9.500000,45.0,0.0,0.00,0.0,50.9
2010-01-02,58.375000,8.6,9.458333,46.0,0.0,0.00,0.0,49.3
2010-01-03,56.708333,14.0,39.583333,43.2,0.0,0.00,0.0,47.0
2010-01-04,66.208333,10.2,22.958333,45.9,3.0,0.05,0.0,54.5
2010-01-05,48.416667,16.1,9.291667,47.9,1.0,0.00,0.0,51.4
...,...,...,...,...,...,...,...,...
2019-12-27,52.833333,22.4,0.250000,47.6,0.0,0.00,0.0,50.6
2019-12-28,59.875000,11.6,9.416667,45.6,0.0,0.00,0.0,49.3
2019-12-29,72.458333,7.2,87.208333,44.9,2.0,0.01,0.0,48.6


In [47]:
# rain

# not_rainy = 0 hours, or  <2.5 mm total rain (Laurina changing to 0.5 inches)
# very_rainy = >=6 hour and >=10mm (Laurina changing to 2 inches)
# lightly_rainy = leftovers
step2_df['not_rainy'] = (step2_df['rain_sum'] <= 0.5) | (step2_df['precipitation_hours'] == 0)
step2_df['not_rainy'] = step2_df['not_rainy'].astype(int)

step2_df['very_rainy'] = (step2_df['rain_sum'] >= 2) | (step2_df['precipitation_hours'] >= 6)
step2_df['very_rainy'] = step2_df['very_rainy'].astype(int)

step2_df['lightly_rainy'] = (step2_df['not_rainy'] == 0) & (step2_df['very_rainy'] == 0)
step2_df['lightly_rainy'] = step2_df['lightly_rainy'].astype(int)

step2_df

,humid_avg,wind_high,cloud_avg,temp_6,precipitation_hours,rain_sum,snowfall_sum,temp_high,not_rainy,very_rainy,lightly_rainy
pure_date,,,,,,,,,,,
2010-01-01,51.916667,8.6,9.500000,45.0,0.0,0.00,0.0,50.9,1,0,0
2010-01-02,58.375000,8.6,9.458333,46.0,0.0,0.00,0.0,49.3,1,0,0
2010-01-03,56.708333,14.0,39.583333,43.2,0.0,0.00,0.0,47.0,1,0,0
2010-01-04,66.208333,10.2,22.958333,45.9,3.0,0.05,0.0,54.5,1,0,0
2010-01-05,48.416667,16.1,9.291667,47.9,1.0,0.00,0.0,51.4,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,52.833333,22.4,0.250000,47.6,0.0,0.00,0.0,50.6,1,0,0
2019-12-28,59.875000,11.6,9.416667,45.6,0.0,0.00,0.0,49.3,1,0,0
2019-12-29,72.458333,7.2,87.208333,44.9,2.0,0.01,0.0,48.6,1,0,0


In [48]:
# freezing

# freezing = >19hours with <32F: Use "temp_6" (6th highest temp of day) as cutoff.
# cold = high <60F
# hot = >6hours with >80F
# warm = leftovers

step2_df['freezing'] = (step2_df['temp_6'] <= 32)
step2_df['freezing'] = step2_df['freezing'].astype(int)

step2_df['cold'] = (step2_df['temp_high'] <= 60) & (step2_df['freezing'] == 0)
step2_df['cold'] = step2_df['cold'].astype(int)

step2_df['hot'] = (step2_df['temp_6'] >= 80)
step2_df['hot'] = step2_df['hot'].astype(int)

step2_df['warm'] = (step2_df['freezing'] == 0) & (step2_df['cold'] == 0) & (step2_df['hot'] == 0)
step2_df['warm'] = step2_df['warm'].astype(int)

step2_df

,humid_avg,wind_high,cloud_avg,temp_6,precipitation_hours,rain_sum,snowfall_sum,temp_high,not_rainy,very_rainy,lightly_rainy,freezing,cold,hot,warm
pure_date,,,,,,,,,,,,,,,
2010-01-01,51.916667,8.6,9.500000,45.0,0.0,0.00,0.0,50.9,1,0,0,0,1,0,0
2010-01-02,58.375000,8.6,9.458333,46.0,0.0,0.00,0.0,49.3,1,0,0,0,1,0,0
2010-01-03,56.708333,14.0,39.583333,43.2,0.0,0.00,0.0,47.0,1,0,0,0,1,0,0
2010-01-04,66.208333,10.2,22.958333,45.9,3.0,0.05,0.0,54.5,1,0,0,0,1,0,0
2010-01-05,48.416667,16.1,9.291667,47.9,1.0,0.00,0.0,51.4,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,52.833333,22.4,0.250000,47.6,0.0,0.00,0.0,50.6,1,0,0,0,1,0,0
2019-12-28,59.875000,11.6,9.416667,45.6,0.0,0.00,0.0,49.3,1,0,0,0,1,0,0
2019-12-29,72.458333,7.2,87.208333,44.9,2.0,0.01,0.0,48.6,1,0,0,0,1,0,0


In [49]:
# snow

# data used: total snow for the day
# not_snowy = 0
# very_snowy = >6"
# lightly_snowy = >0
# (unlike rain, we ignore hours)
    
step2_df['not_snowy'] = (step2_df['snowfall_sum'] == 0)
step2_df['not_snowy'] = step2_df['not_snowy'].astype(int)

step2_df['very_snowy'] = (step2_df['snowfall_sum'] >= 6)
step2_df['very_snowy'] = step2_df['very_snowy'].astype(int)

step2_df['lightly_snowy'] = (step2_df['not_snowy'] == 0) & (step2_df['very_snowy'] == 0)
step2_df['lightly_snowy'] = step2_df['lightly_snowy'].astype(int)

step2_df

,humid_avg,wind_high,cloud_avg,temp_6,precipitation_hours,rain_sum,snowfall_sum,temp_high,not_rainy,very_rainy,lightly_rainy,freezing,cold,hot,warm,not_snowy,very_snowy,lightly_snowy
pure_date,,,,,,,,,,,,,,,,,,
2010-01-01,51.916667,8.6,9.500000,45.0,0.0,0.00,0.0,50.9,1,0,0,0,1,0,0,1,0,0
2010-01-02,58.375000,8.6,9.458333,46.0,0.0,0.00,0.0,49.3,1,0,0,0,1,0,0,1,0,0
2010-01-03,56.708333,14.0,39.583333,43.2,0.0,0.00,0.0,47.0,1,0,0,0,1,0,0,1,0,0
2010-01-04,66.208333,10.2,22.958333,45.9,3.0,0.05,0.0,54.5,1,0,0,0,1,0,0,1,0,0
2010-01-05,48.416667,16.1,9.291667,47.9,1.0,0.00,0.0,51.4,1,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,52.833333,22.4,0.250000,47.6,0.0,0.00,0.0,50.6,1,0,0,0,1,0,0,1,0,0
2019-12-28,59.875000,11.6,9.416667,45.6,0.0,0.00,0.0,49.3,1,0,0,0,1,0,0,1,0,0
2019-12-29,72.458333,7.2,87.208333,44.9,2.0,0.01,0.0,48.6,1,0,0,0,1,0,0,1,0,0


In [50]:
# wind

# windy = maximum	> ?? mph
# maybe 8? mayb 12
    
step2_df['windy'] = (step2_df['wind_high'] >= 12)
step2_df['windy'] = step2_df['windy'].astype(int)

step2_df

,humid_avg,wind_high,cloud_avg,temp_6,precipitation_hours,rain_sum,snowfall_sum,temp_high,not_rainy,very_rainy,lightly_rainy,freezing,cold,hot,warm,not_snowy,very_snowy,lightly_snowy,windy
pure_date,,,,,,,,,,,,,,,,,,,
2010-01-01,51.916667,8.6,9.500000,45.0,0.0,0.00,0.0,50.9,1,0,0,0,1,0,0,1,0,0,0
2010-01-02,58.375000,8.6,9.458333,46.0,0.0,0.00,0.0,49.3,1,0,0,0,1,0,0,1,0,0,0
2010-01-03,56.708333,14.0,39.583333,43.2,0.0,0.00,0.0,47.0,1,0,0,0,1,0,0,1,0,0,1
2010-01-04,66.208333,10.2,22.958333,45.9,3.0,0.05,0.0,54.5,1,0,0,0,1,0,0,1,0,0,0
2010-01-05,48.416667,16.1,9.291667,47.9,1.0,0.00,0.0,51.4,1,0,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,52.833333,22.4,0.250000,47.6,0.0,0.00,0.0,50.6,1,0,0,0,1,0,0,1,0,0,1
2019-12-28,59.875000,11.6,9.416667,45.6,0.0,0.00,0.0,49.3,1,0,0,0,1,0,0,1,0,0,0
2019-12-29,72.458333,7.2,87.208333,44.9,2.0,0.01,0.0,48.6,1,0,0,0,1,0,0,1,0,0,0


In [51]:
# humidity

# average hourlies, then:
# low_humidity = <30%
# high_humidity = >60%
# medium_humidity = leftovers

step2_df['low_humidity'] = (step2_df['humid_avg'] <= 30)
step2_df['low_humidity'] = step2_df['low_humidity'].astype(int)

step2_df['high_humidity'] = (step2_df['humid_avg'] >= 60)
step2_df['high_humidity'] = step2_df['high_humidity'].astype(int)

step2_df['medium_humidity'] = (step2_df['low_humidity'] == 0) & (step2_df['high_humidity'] == 0)
step2_df['medium_humidity'] = step2_df['medium_humidity'].astype(int)

step2_df

,humid_avg,wind_high,cloud_avg,temp_6,precipitation_hours,rain_sum,snowfall_sum,temp_high,not_rainy,very_rainy,...,cold,hot,warm,not_snowy,very_snowy,lightly_snowy,windy,low_humidity,high_humidity,medium_humidity
pure_date,,,,,,,,,,,,,,,,,,,,,
2010-01-01,51.916667,8.6,9.500000,45.0,0.0,0.00,0.0,50.9,1,0,...,1,0,0,1,0,0,0,0,0,1
2010-01-02,58.375000,8.6,9.458333,46.0,0.0,0.00,0.0,49.3,1,0,...,1,0,0,1,0,0,0,0,0,1
2010-01-03,56.708333,14.0,39.583333,43.2,0.0,0.00,0.0,47.0,1,0,...,1,0,0,1,0,0,1,0,0,1
2010-01-04,66.208333,10.2,22.958333,45.9,3.0,0.05,0.0,54.5,1,0,...,1,0,0,1,0,0,0,0,1,0
2010-01-05,48.416667,16.1,9.291667,47.9,1.0,0.00,0.0,51.4,1,0,...,1,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,52.833333,22.4,0.250000,47.6,0.0,0.00,0.0,50.6,1,0,...,1,0,0,1,0,0,1,0,0,1
2019-12-28,59.875000,11.6,9.416667,45.6,0.0,0.00,0.0,49.3,1,0,...,1,0,0,1,0,0,0,0,0,1
2019-12-29,72.458333,7.2,87.208333,44.9,2.0,0.01,0.0,48.6,1,0,...,1,0,0,1,0,0,0,0,1,0


In [52]:
# clouds

# average hourly "percent cloud cover"
# clear = <=33%
# cloudy = >= 67%
# partly_cloudy = leftovers

step2_df['clear'] = (step2_df['cloud_avg'] <= 30)
step2_df['clear'] = step2_df['clear'].astype(int)

step2_df['cloudy'] = (step2_df['cloud_avg'] >= 60)
step2_df['cloudy'] = step2_df['cloudy'].astype(int)

step2_df['partly_cloudy'] = (step2_df['clear'] == 0) & (step2_df['cloudy'] == 0)
step2_df['partly_cloudy'] = step2_df['partly_cloudy'].astype(int)

step2_df

,humid_avg,wind_high,cloud_avg,temp_6,precipitation_hours,rain_sum,snowfall_sum,temp_high,not_rainy,very_rainy,...,not_snowy,very_snowy,lightly_snowy,windy,low_humidity,high_humidity,medium_humidity,clear,cloudy,partly_cloudy
pure_date,,,,,,,,,,,,,,,,,,,,,
2010-01-01,51.916667,8.6,9.500000,45.0,0.0,0.00,0.0,50.9,1,0,...,1,0,0,0,0,0,1,1,0,0
2010-01-02,58.375000,8.6,9.458333,46.0,0.0,0.00,0.0,49.3,1,0,...,1,0,0,0,0,0,1,1,0,0
2010-01-03,56.708333,14.0,39.583333,43.2,0.0,0.00,0.0,47.0,1,0,...,1,0,0,1,0,0,1,0,0,1
2010-01-04,66.208333,10.2,22.958333,45.9,3.0,0.05,0.0,54.5,1,0,...,1,0,0,0,0,1,0,1,0,0
2010-01-05,48.416667,16.1,9.291667,47.9,1.0,0.00,0.0,51.4,1,0,...,1,0,0,1,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,52.833333,22.4,0.250000,47.6,0.0,0.00,0.0,50.6,1,0,...,1,0,0,1,0,0,1,1,0,0
2019-12-28,59.875000,11.6,9.416667,45.6,0.0,0.00,0.0,49.3,1,0,...,1,0,0,0,0,0,1,1,0,0
2019-12-29,72.458333,7.2,87.208333,44.9,2.0,0.01,0.0,48.6,1,0,...,1,0,0,0,0,1,0,0,1,0


## finally, get summary statistics

### Beware, beware: for final product, need to add filtering by dates!

In [53]:
step2_df.columns

Index(['humid_avg', 'wind_high', 'cloud_avg', 'temp_6', 'precipitation_hours',
       'rain_sum', 'snowfall_sum', 'temp_high', 'not_rainy', 'very_rainy',
       'lightly_rainy', 'freezing', 'cold', 'hot', 'warm', 'not_snowy',
       'very_snowy', 'lightly_snowy', 'windy', 'low_humidity', 'high_humidity',
       'medium_humidity', 'clear', 'cloudy', 'partly_cloudy'],
      dtype='object')

In [54]:
# list of column names from step2_df to be aggregated
bool_cols = ['freezing', 'cold', 'warm', 'hot',
             'not_rainy', 'lightly_rainy', 'very_rainy',
             'not_snowy', 'lightly_snowy', 'very_snowy',
             'windy',
             'low_humidity', 'medium_humidity', 'high_humidity',
             'clear', 'partly_cloudy', 'cloudy']

# initialize output dataframe
blank_dict = {stat:[] for stat in bool_cols}
blank_dict['geo'] = []

hist_stats_df = pd.DataFrame(blank_dict)

row_count = len(step2_df)


In [55]:
# history_stats_list = []

# for stat in bool_cols:
#     history_stats_list.append( (stat, step2_df[stat].sum() / row-count )

history_stats_dict = {}

for stat in bool_cols:
    history_stats_dict[stat] = [step2_df[stat].sum() / row_count]

history_stats_dict['geo'] = geo_input


In [56]:
# makes a one-row dataframe with all we need
hist_stats_row = pd.DataFrame(history_stats_dict)
# add it as a row to the output df
hist_stats_df = pd.concat([hist_stats_df, hist_stats_row])

In [57]:
hist_stats_df

,freezing,cold,warm,hot,not_rainy,lightly_rainy,very_rainy,not_snowy,lightly_snowy,very_snowy,windy,low_humidity,medium_humidity,high_humidity,clear,partly_cloudy,cloudy,geo
0,0.0,0.317634,0.496166,0.186199,0.909912,0.0,0.286145,0.97563,0.021358,0.003012,0.515334,0.000274,0.168127,0.831599,0.374589,0.292442,0.332968,jp | Tokyo


In [58]:
hist_stats_df.to_csv('five_cities.csv', mode='a', index=False, header=False)